# Learn to Develop and Customize AI Workflows with Flows: AMLD Setup and Test Run Notebook

Thank you for your interest in our workshop!

Please download this notebook and execute all the cells. If you encounter any issues, please reach out to us.

### Environment

Before proceeding, ensure you have the following installed:
- Python 3.10 or later
- Any setup which allows running a notebook (e.g., Jupyter Notebook, JupyterLab)

If you have conda installed on your machine, you can use the following instructions:

```unix
 conda create -n amld python=3.11 --yes
 conda activate amld
 conda install -c conda-forge jupyterlab --yes
 conda install ipykernel --yes
 python -m ipykernel install --user --name amld --display-name "amld"
 pip install --upgrade ipywidgets
 jupyter lab
```

Note: The setup has been tested on Unix-like systems such as macOS and Linux.

---

Most of the Flows we're going to develop during the workshop will require access to LLMs. Therefore, once you make sure that the notebook executes succesfully, please ensure you possess an **OpenAI API key** (check out [this link](https://platform.openai.com/docs/quickstart/account-setup) on how to get one) or a key from a provider supported by LiteLLM (check the list of supported providers [here]((https://docs.litellm.ai/docs/providers))). We highly recommend using an OpenAI key for this workshop. 

Also plese join our Discord server where we will be responding to questions asyncronously as well: [https://discord.gg/pFQTddUbjK](https://discord.gg/pFQTddUbjK)

Don't worry if you do not understand the code in this notebook! After the workshop you'll know how to customize AI workflows on effectively!


## 1. Installing aiFlows

In [1]:
!pip install aiflows -U

## 2. Imports

In [1]:
from IPython.core.magic import register_cell_magic
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
from omegaconf import OmegaConf
import os
from aiflows.utils import colink_utils, serving
from aiflows.utils import logging
logging.set_verbosity_info()

# Some useful functions 

@register_cell_magic
def compile_and_writefile(line, cell):
    # Compile the code in the cell
    compiled_code = compile(cell, '<string>', 'exec')
    #check if all directories exist and create them if they don't   
    # Extract the directory path from the file path
    directory = os.path.dirname(line)

    # Check if the directory exists
    if not os.path.exists(directory):
        # Create the directory if it doesn't exist
        os.makedirs(directory)
        # Write the compiled code to a file
    with open(line, 'w') as f:
        f.write(cell)        

def dict_to_yaml(dictionary, output_file):
    """
    Convert a dictionary to YAML using OmegaConf and write to a file.

    :param dictionary: Dictionary to convert.
    :type dictionary: dict
    :param output_file: Path to the output YAML file.
    :type output_file: str
    """
    # Convert dictionary to OmegaConf config object
    config = OmegaConf.create(dictionary)

    # Write the config object to the output YAML file
    OmegaConf.save(config, output_file)

/Users/nicolasbaldwin/opt/miniconda3/envs/tyst/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. Defining a Toy Flow

In [2]:
%%compile_and_writefile ReverseNumberFlowModule/ReverseNumberAtomicFlow.py

from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage

class ReverseNumberAtomicFlow(AtomicFlow):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    # Customize the logic within this function as needed for your specific flow requirements.
    def run(self, input_message: FlowMessage):

        #Get data dictionary from input message
        input_data = input_message.data
        
        #get input number from data dictionary (int)
        input_number = input_data["number"]
        
        #TODO: reverse the input number (e.g. 1234 -> 4321)
        reversed_number = int(str(input_number)[::-1])
        
        #Create response dictionary
        response = {"reversed_number": reversed_number}
        
        #package ouput message to send back
            #This method packages `response` in a FlowMessage object 
            # containing the necessary metadata to send the message back
            # to the sender of the input message. 
        reply = self.package_output_message(
            input_message=input_message,
            response=response,
        )
        
        #send back reply
        self.send_message(
            reply
        )

In [3]:
default_config_reverse_number = \
{
    "name": "ReverseNumber",
    "description": "A flow that takes in a number and reverses it.",

    "_target_": "ReverseNumberFlowModule.ReverseNumberAtomicFlow.ReverseNumberAtomicFlow.instantiate_from_default_config",

    "input_interface": "number",
    "output_interface": "reversed_number",
}

dict_to_yaml(default_config_reverse_number, "ReverseNumberFlowModule/ReverseNumberAtomicFlow.yaml")

## 3. Setting up the Infrastructure

In [4]:
# Starting CoLink server
cl = colink_utils.start_colink_server()

In [5]:
# Serving the Flow
serving.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)

[2024-04-08 10:35:38,386][aiflows.utils.serving:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
[2024-04-08 10:35:38,387][aiflows.utils.serving:117][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:35:38,388][aiflows.utils.serving:118][INFO] - parallel_dispatch: False
[2024-04-08 10:35:38,389][aiflows.utils.serving:119][INFO] - singleton: True



True

In [6]:
# Start a worker thread to handle incoming messages
run_dispatch_worker_thread(cl)

[2024-04-08 10:35:39,857][aiflows.workers.dispatch_worker:236][INFO] - Dispatch worker started in attached thread.
[2024-04-08 10:35:39,859][aiflows.workers.dispatch_worker:237][INFO] - dispatch_point: coflows_dispatch


In [7]:
# Getting an instance of the flow
proxy_reverse_number_flow = serving.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_atomic",
    user_id="local",
)

[2024-04-08 10:35:41,483][aiflows.utils.serving:336][INFO] - Mounted b0ee3388-91ad-4208-9edb-98f8f4733467 at flows:reverse_number_atomic:mounts:local:b0ee3388-91ad-4208-9edb-98f8f4733467


## 4. Running the Flow

In [8]:
input_data = {"number": 12345}

# Package your data in a Flow Message
input_message = proxy_reverse_number_flow.package_input_message(input_data)

# Send a message to reverse number and ask to get an answer back in a future

future = proxy_reverse_number_flow.get_reply_future(input_message)

# Get the response from the future
# To get the response as a data dictionary
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

[2024-04-08 10:35:43,714][aiflows.workers.dispatch_worker:119][INFO] - 
~~~ Dispatch task ~~~
[2024-04-08 10:35:43,734][aiflows.workers.dispatch_worker:161][INFO] - flow_endpoint: reverse_number_atomic
[2024-04-08 10:35:43,735][aiflows.workers.dispatch_worker:162][INFO] - flow_id: b0ee3388-91ad-4208-9edb-98f8f4733467
[2024-04-08 10:35:43,736][aiflows.workers.dispatch_worker:163][INFO] - owner_id: local
[2024-04-08 10:35:43,737][aiflows.workers.dispatch_worker:164][INFO] - message_paths: ['push_tasks:4cda6af3-fc6a-4428-8221-6341c1f88595:msg']
[2024-04-08 10:35:43,738][aiflows.workers.dispatch_worker:165][INFO] - parallel_dispatch: False

[2024-04-08 10:35:43,798][aiflows.workers.dispatch_worker:188][INFO] - Input message source: Proxy_reverse_number_atomic


Data sent:
 {'number': 12345} 

REPLY:
 {'reversed_number': 54321} 



## You're ready!